In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import networkx as nx
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from openqaoa.qaoa_parameters import QAOACircuitParams, create_qaoa_variational_params, Hamiltonian
from openqaoa.devices import create_device
from openqaoa.backends.qaoa_backend import get_qaoa_backend
from openqaoa.optimizers import get_optimizer
from openqaoa.utilities import *
from openqaoa.workflows.optimizer import QAOA, RQAOA
from openqaoa.problems.problem import QUBO, MaximumCut, MinimumVertexCover

In [ ]:
## with weights there's a difference in the ground state energy between the hamiltonian from the maxcut qubo and the hamiltonian from the graph 
### simply because single terms don't matter in the maxcut, i.e. different hamiltonina, i.e. offset

In [ ]:
## note that the function get_qubo_problem ran outside the class with g gives the same terms. The problem is that the graph inside gets relabeled
###  self._G = nx.relabel_nodes(input_networkx_graph, mapping)

In [ ]:
mapping = dict(
            zip(g, range(
                g.number_of_nodes()))
        )

In [ ]:
mapping

In [ ]:
g.number_of_nodes()

In [ ]:
plot_graph(MaximumCut(g).G)

### MaxCut

In [ ]:
g = random_k_regular_graph(degree=3, nodes=range(8), weighted=True, biases=True)  # without biases the energy is the same, with is different
print('Graph edges', g.edges)
plot_graph(g)
#plt.savefig("graph_sigma={}.png".format(sigma))
#plt.clf()


# Define the problem and translate it into a binary Qubo.
maxcut_prob = MaximumCut(g)
maxcut_qubo = maxcut_prob.get_qubo_problem()
print("QUBO terms", maxcut_qubo.terms)

# Define Hamiltonian
cost_hamiltonian_graph = hamiltonian_from_graph(g)
print('Hamiltonian from graph', cost_hamiltonian_graph)
energy_graph, configuration_graph = ground_state_hamiltonian(cost_hamiltonian_graph)
print(energy_graph, configuration_graph)

print('Hamiltonian from QUBO', maxcut_qubo.hamiltonian)
cost_hamiltonian_qubo = maxcut_qubo.hamiltonian
energy_qubo, configuration_qubo = ground_state_hamiltonian(cost_hamiltonian_qubo)
print(energy_qubo, configuration_qubo)


In [ ]:
g.edges(data="weight")

In [ ]:
# code from openqaoa/problems/problem.py get_qubo_problem(self):

terms = []
weights = []

for u, v, edge_weight in g.edges(data="weight"):
    terms.append([u, v])

    # We expect the edge weight to be given in the attribute called
    # "weight". If it is None, assume a weight of 1.0
    weights.append(
        edge_weight if edge_weight else 100
    )
    
print(terms)  # terms are fine, i.e. they match the graph

qubo_new = QUBO(g.number_of_nodes(), terms, weights) 

In [ ]:
qubo_new.terms # that also matches the graph

In [ ]:
qubo_new.weights

In [ ]:
ground_state_hamiltonian(qubo_new.hamiltonian)

In [ ]:
hamiltonian_from_graph(g).terms

In [ ]:
ground_state_hamiltonian(hamiltonian_from_graph(g))

In [ ]:
print("QUBO terms", qubo_new.terms)

# Define Hamiltonian
cost_hamiltonian_graph = hamiltonian_from_graph(g)  # bug must be here
print('Hamiltonian from graph', cost_hamiltonian_graph)  
energy_graph, configuration = ground_state_hamiltonian(cost_hamiltonian_graph)
print(energy_graph, configuration)

print('Hamiltonian from QUBO', qubo_new.hamiltonian) # bug must be here this is fine and it amtches the graph too!
cost_hamiltonian_qubo = qubo_new.hamiltonian
energy_qubo, configuration = ground_state_hamiltonian(cost_hamiltonian_qubo)
print(energy_qubo, configuration)

In [ ]:
cost_hamiltonian_graph.qubits_singles

In [ ]:
cost_hamiltonian_qubo.qubits_singles

In [ ]:
cost_hamiltonian_graph.expression

In [ ]:
cost_hamiltonian_qubo.expression

### Minimum Vertex Cover

In [ ]:
g = random_k_regular_graph(degree=3, nodes=range(6), weighted=False, biases=False)
plot_graph(g)
#plt.savefig("graph_sigma={}.png".format(sigma))
#plt.clf()

# Define te problem and translate it into a binary Qubo.

vc = MinimumVertexCover(G,field = field,penalty = penalty).get_qubo_problem()
print(vc.terms)

# Define Hamiltonian
cost_hamiltonian = hamiltonian_from_graph(g)
print(cost_hamiltonian)


# Solve the problem analytically
energy, configuration = ground_state_hamiltonian(cost_hamiltonian)
